<a href="https://colab.research.google.com/github/ZKingQ/CS598-DLH-SP24/blob/main/DLH_Team_71_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS598 Deep Learning for Healthcare - Final Project - DeepMicro: Deep Representation Learning for Disease Prediction based on Microbiome Data

### Lotte Zhu, Kaiqing Zhang, Matthew Trueblood (Team ID: 71)
#### GitHub Link: https://github.com/ZKingQ/CS598-DLH-SP24

# Data loading

We will load and unzip the data from where it is hosted on a public cloud (google drive)

In [3]:
import os
import requests
from zipfile import ZipFile
from io import BytesIO

def fetch_data_from_remote():
    # cloud location where the .zip file of the data used by this project is stored
    url = 'https://drive.google.com/file/d/1c6ZPVp346e1wQH6opg4YwlCRjRgfTXCk/view?usp=drive_link'
    
    file_id = url.split('/')[-2]
    dwn_url = 'https://drive.google.com/uc?export=download&id=' + file_id
    
    # unzip the data into the local working directory
    response = requests.get(dwn_url)
    zip_content = BytesIO(response.content)
    with ZipFile(zip_content, 'r') as main_zip:
        main_zip.extractall()
        with main_zip.open('data/abundance.zip') as nested_zip_abundance:
            with ZipFile(nested_zip_abundance) as zip_ref_abundance:
                zip_ref_abundance.extractall('data/abundance')
    
        with main_zip.open('data/marker.zip') as nested_zip_marker:
            with ZipFile(nested_zip_marker) as zip_ref_marker:
                zip_ref_marker.extractall('data/marker')
                
if os.path.exists('data'):
    print("Data already exists locally")
else:
    fetch_data_from_remote()
    print("Data fetched and unzipped successfully")

# show that the data is available locally now
!ls data/abundance/
!ls data/marker/

Data already exists locally
abundance_Cirrhosis.csv  abundance_IBD.txt        abundance_WT2D.txt
abundance_Cirrhosis.txt  abundance_Obesity.txt
abundance_Colorectal.txt abundance_T2D.txt
marker_Cirrhosis.txt  marker_IBD.txt        marker_T2D.txt
marker_Colorectal.txt marker_Obesity.txt    marker_WT2D.txt


In [ ]:
label_dict = {
    # Controls
    'n': 0,
    # Chirrhosis
    'cirrhosis': 1,
    # Colorectal Cancer
    'cancer': 1, 'small_adenoma': 0,
    # IBD
    'ibd_ulcerative_colitis': 1, 'ibd_crohn_disease': 1,
    # T2D and WT2D
    't2d': 1,
    # Obesity
    'leaness': 0, 'obesity': 1,
}
def loadData(data_dir, filename, dtype=None):
    feature_string = ''
    if filename.split('_')[0] == 'abundance':
        feature_string = "k__"
    if filename.split('_')[0] == 'marker':
        feature_string = "gi|"
    # read file
    filename = data_dir + filename
    if not os.path.isfile(filename):
        print("FileNotFoundError: File {} does not exist".format(filename))
        exit()
    raw = pd.read_csv(filename, sep='\t', index_col=0, header=None)

    # select rows having feature index identifier string
    X = raw.loc[raw.index.str.contains(feature_string, regex=False)].T

    # get class labels
    Y = raw.loc['disease']
    Y = Y.replace(label_dict)

    # train and test split
    X_train, X_test, y_train, y_test = train_test_split(X.values.astype(dtype), Y.values.astype('int'), test_size=0.2, random_state=seed, stratify=Y.values)
    print("Train data shape: ", X_train.shape)
    print("Test data shape: ", X_test.shape)
    return X_train, X_test, y_train, y_test

# Introduction
<!-- This is an introduction to your report, you should edit this text/mardown section to compose. In this text/markdown, you should introduce:

*   Background of the problem
  * what type of problem: disease/readmission/mortality prediction,  feature engineeing, data processing, etc
  * what is the importance/meaning of solving the problem
  * what is the difficulty of the problem
  * the state of the art methods and effectiveness.
*   Paper explanation
  * what did the paper propose
  * what is the innovations of the method
  * how well the proposed method work (in its own metrics)
  * what is the contribution to the reasearch regime (referring the Background above, how important the paper is to the problem). -->


The expanding knowledge of microbiota uncovers its crucial role in human health [1]. It plays an important role in immune system, metabolism functions and even carcinognesis of certain cancers, hence microbiota can be used to predict various disease with emerging sequencing technologies [1,2,3,4,5,6]. However, there are three major challenges to realize the predictions in practice [7]. First, the low number of samples, together with the large number of features, leads to the curse of dimensionality. Second, there is a research gap in using strain-level profiles to classify samples into patient and healthy control groups across different diseases. Third, a rigorous validation framework is essential. Prior research has shown that tuning hyperparameters on the test set without a separate validation set may lead to an overestimation of model performance [8,9,10].

The DeepMicro paper proposes the deployment of autoencoders to learn low-dimensional representations from microbiota data and then predict disease by another classification model based on the learned representations, with both trainings having thorough validation schemes [7]. The authors hypothesize that these innovations can contribute to the followings:

1. The appropriate autoencoders can effectively solve the curse of dimensionality.

2. They also reduce latency compared with alternative models without representation learing, in the mean time maintaining favorable training metrics.

In this project, our primary objective revolves around the implementation and evaluation of various autoencoder architectures. We aim to validate the disease prediction capabilities as outlined in the paper. Furthermore, we discuss the ablations inherent in the innovation of autoencoder. The rationale for choosing this paper stems from our pursuit of knowledge in deep representation learning.


# Scope of Reproducibility:

<!-- List hypotheses from the paper you will test and the corresponding experiments you will run.


1.   Hypothesis 1: xxxxxxx
2.   Hypothesis 2: xxxxxxx

You can insert images in this notebook text, [see this link](https://stackoverflow.com/questions/50670920/how-to-insert-an-inline-image-in-google-colaboratory-from-google-drive) and example below:

![sample_image.png](https://drive.google.com/uc?export=view&id=1g2efvsRJDxTxKz-OY3loMhihrEUdBxbc)


You can also use code to display images, see the code below.

The images must be saved in Google Drive first.

-->

Our team achieved successful replication of the model by leveraging its open source codebase, resulting in metrics that are comparable to those showcased in the DeepMicro paper. In our endeavor, we thoroughly examined and addressed the claims below put forth in the original paper.

1. **Dimensionality reduction engineering with traditional statistical techniques including Principal Component Analysis (PCA) and Gaussian Random Projection (GRP)**

- Principal Component Analysis (PCA) aims to capture the most significant patterns and variations in a dataset by identifying orthogonal axes, known as principal components, that maximize the data variance.
- Gaussian Random Projection (GRP) seeks to preserve pairwise distances between data points by projecting them onto a lower-dimensional space using random projections drawn from a Gaussian distribution.

2. **Innovated representation learning employing with four different autoencoders including Shallow Autoencoder (SAE), Deep Autoencoder (DAE), Variational Autoencoder (VAE), and Convolutional Autoencoder (CAE)**

- Shallow Autoencoder (SAE): This is the simplest form of an autoencoder, consisting of a fully
connected encoder layer and a decoder layer. The latent representation is obtained from the encoder
layer, which is a lower-dimensional space compared to the original input.
- Deep Autoencoder (DAE): In addition to the encoder and decoder layers, DAE introduces hidden
layers between the input and latent layers and between the latent and output layers. Rectified Linear
Unit (ReLU) activation functions are used in the hidden layers.
- Variational Autoencoder (VAE): VAE learns probabilistic representations by approximating the
true posterior distribution of latent embeddings. It assumes that the posterior distribution follows
a Gaussian distribution. VAE uses an encoder network to encode the means and variances of the
Gaussian distribution and samples the latent representation from this distribution. The decoder
network then reconstructs the input based on the sampled latent representation.
- Convolutional Autoencoder (CAE): Instead of fully connected layers, CAE incorporates convo-
lutional layers, where each unit is connected to local regions of the previous layer. Convolutional layers use filters (kernels) to perform convolution operations. CAE employs convolutional transpose layers (deconvolutional layers) to make the decoder symmetric to the encoder. No pooling layers
are used in CAE.

3. **Classification learning including including Support Vector Machine (SVM), Random Forest (RF), and Multi-Layer Perceptron (MLP)**
- Support Vector Machine (SVM) is a supervised learning algorithm that aims to find an optimal hyperplane to classify data by maximizing the margin between different classes.
- Random Forest (RF) is an ensemble learning method that constructs a multitude of decision trees and combines their predictions to make classifications.
- Multi-Layer Perceptron (MLP) is a type of neural network that consists of multiple layers of interconnected nodes, enabling it to learn non-linear relationships and perform classification tasks.


# Methodology

This methodology is the core of your project. It consists of run-able codes with necessary annotations to show the expeiment you executed for testing the hypotheses.

The methodology at least contains two subsections **data** and **model** in your experiment.

## Environment Set Up and Packages Import

In [2]:
# Python version
!python --version

Python 3.10.12


In [5]:
# import packages needed in this project
import numpy as np
import pandas as pd
import time
import json
import datetime
import math
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.random_projection import GaussianRandomProjection
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# importing PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

##  Data
Data includes raw data (MIMIC III tables), descriptive statistics (our homework questions), and data processing (feature engineering).
  * Source of the data: where the data is collected from; if data is synthetic or self-generated, explain how. If possible, please provide a link to the raw datasets.
  * Statistics: include basic descriptive statistics of the dataset like size, cross validation split, label distribution, etc.
  * Data process: how do you munipulate the data, e.g., change the class labels, split the dataset to train/valid/test, refining the dataset.
  * Illustration: printing results, plotting figures for illustration.
  * You can upload your raw dataset to Google Drive and mount this Colab to the same directory. If your raw dataset is too large, you can upload the processed dataset and have a code to load the processed dataset.

### Data Path

We downloaded two datasets, abundance and marker from the [DeepMicro codebase](https://github.com/minoh0201/DeepMicro/tree/master/data). They are stored in the following path after you've downloaded them.

In [6]:
# data dir
raw_data_dir = 'data/'

### Data Description

Our reproductivity utilizes the same datasets as the original paper, which include six disease (Table 1). They are inflammatory bowel disease (IBD), type 2 diabetes in European women (EW-T2D), type 2 diabetes in Chinese (C-T2D), obesity (Obesity), liver cirrhosis (Cirrhosis), and colorectal cancer (Colorectal).
![](assets/Data_Table1.png)

In each dataset, marker profile and abundance profile of microbiome are used to train our models (Table 2).
![](assets/Data_Table2.png)

All the data are stored in txt format and the data path structure is as following.
```
.
├── ...
├── data                              # Data folder
│   ├── marker                        # Marker profile data
│       ├── marker_IBD.txt            # Inflammatory bowel disease (IBD)
│       ├── marker_WT2D.txt           # Type 2 diabetes in European women (EW-T2D)
│       ├── marker_T2D.txt            # Type 2 diabetes in Chinese (C-T2D)
│       ├── marker_Obesity.txt        # Obesity (Obesity)
│       ├── marker_Cirrhosis.txt      # Liver cirrhosis (Cirrhosis)
│       ├── marker_Colorectal.txt     # Colorectal cancer (Colorectal)
│   ├── abundance                     # Abundance profile data
│       ├── abundance_IBD.txt         # Inflammatory bowel disease (IBD)
│       ├── abundance_WT2D.txt        # Type 2 diabetes in European women (EW-T2D)
│       ├── abundance_T2D.txt         # Type 2 diabetes in Chinese (C-T2D)
│       ├── abundance_Obesity.txt     # Obesity (Obesity)
│       ├── abundance_Cirrhosis.txt   # Liver cirrhosis (Cirrhosis)
│       ├── abundance_Colorectal.txt  # Colorectal cancer (Colorectal)
└── ...
```
In each txt file, it has different number of features and data points as shown in Table 1 and Table 2 above. In the sample demonstration below. The colorectal cancer of microbiome abundance profile has 503 features (exluding those dummy labels) and 121 entries.



In [7]:
# Extract a sample data of colorectal cancer of microbiome abundance profile
sample = pd.read_csv(os.path.join(raw_data_dir, "abundance", "abundance_Colorectal.txt"), sep='\t', index_col=0, header=None)
sample = sample.T
sample.head(10)

,dataset_name,sampleID,subjectID,bodysite,disease,age,gender,country,sequencing_technology,pubmedid,...,k__Eukaryota|p__Ascomycota|c__Saccharomycetes|o__Saccharomycetales|f__Saccharomycetaceae|g__Eremothecium|s__Eremothecium_unclassified,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Lactobacillaceae|g__Lactobacillus|s__Lactobacillus_antri,k__Bacteria|p__Firmicutes|c__Bacilli|o__Bacillales|f__Bacillaceae|g__Lysinibacillus|s__Lysinibacillus_fusiformis,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobacterium|s__Methanobacterium_unclassified,k__Bacteria|p__Firmicutes|c__Bacilli|o__Bacillales|f__Bacillaceae|g__Lysinibacillus|s__Lysinibacillus_boronitolerans,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Enterococcaceae|g__Bavariicoccus|s__Bavariicoccus_seileri,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Enterococcaceae|g__Enterococcus|s__Enterococcus_gilvus,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Lactobacillaceae|g__Lactobacillus|s__Lactobacillus_otakiensis,k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Peptococcaceae|g__Desulfotomaculum|s__Desulfotomaculum_ruminis,k__Bacteria|p__Firmicutes|c__Negativicutes|o__Selenomonadales|f__Veillonellaceae|g__Megasphaera|s__Megasphaera_sp_BV3C16_1
1,Zeller_fecal_colorectal_cancer,CCIS00146684ST-4-0,fr-726,stool,n,72,female,france,Illumina,25432777,...,0,0,0,0,0,0,0,0,0,0
2,Zeller_fecal_colorectal_cancer,CCIS00281083ST-3-0,fr-060,stool,n,53,male,france,Illumina,25432777,...,0,0,0,0,0,0,0,0,0,0
3,Zeller_fecal_colorectal_cancer,CCIS02124300ST-4-0,fr-568,stool,n,35,male,france,Illumina,25432777,...,0,0,0,0,0,0,0,0,0,0
4,Zeller_fecal_colorectal_cancer,CCIS02379307ST-4-0,fr-828,stool,cancer,67,male,france,Illumina,25432777,...,0,0,0,0,0,0,0,0,0,0
5,Zeller_fecal_colorectal_cancer,CCIS03473770ST-4-0,fr-192,stool,n,29,male,france,Illumina,25432777,...,0,0,0,0,0,0,0.03756,0,0,0
6,Zeller_fecal_colorectal_cancer,CCIS06260551ST-3-0,fr-200,stool,cancer,58,male,france,Illumina,25432777,...,0,0,0.31121,0,0.03562,0,0,0,0,0
7,Zeller_fecal_colorectal_cancer,CCIS07539127ST-4-0,fr-460,stool,n,77,female,france,Illumina,25432777,...,0,0,0,0,0,0,0,0,0,0
8,Zeller_fecal_colorectal_cancer,CCIS07648107ST-4-0,fr-053,stool,n,62,female,france,Illumina,25432777,...,0,0,0,0,0,0,0,0,0,0
9,Zeller_fecal_colorectal_cancer,CCIS08668806ST-3-0,fr-214,stool,small_adenoma,63,male,france,Illumina,25432777,...,0,0,0,0,0,0,0,0,0,0
10,Zeller_fecal_colorectal_cancer,CCIS09568613ST-4-0,fr-400,stool,n,67,male,france,Illumina,25432777,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Count the features excluding patients information
len([ _ for _ in sample.columns if '|' in _ ])

503

In [8]:
# Calculate the dimensions of the sameple dataset
sample.shape

(121, 714)

In [9]:
# Have a generate descrption of the sample dataset
sample.describe().T

,count,unique,top,freq
0,,,,
dataset_name,121,1,Zeller_fecal_colorectal_cancer,121
sampleID,121,121,CCIS00146684ST-4-0,1
subjectID,121,121,fr-726,1
bodysite,121,1,stool,121
disease,121,3,cancer,48
...,...,...,...,...
k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Enterococcaceae|g__Bavariicoccus|s__Bavariicoccus_seileri,121,2,0,120
k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Enterococcaceae|g__Enterococcus|s__Enterococcus_gilvus,121,5,0,117
k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Lactobacillaceae|g__Lactobacillus|s__Lactobacillus_otakiensis,121,2,0,120


### Data Process

According to the original paper, the data provided in the codebase have been preprocessed and cleaned properly. We only need to extract the labels columns with feature index indentifier to get `X`. Also, we get `Y` from the `disease` column. This step is implemented in the `loadData` function of class `DeepMiccrobiome` in the Model session below.




### Data Use

Given the running time of dimensionality reduction is long, we choose to only use the first 10 features of each dataset in order to control the total latency in this draft project.

##   Model
The model includes the model definitation which usually is a class, model training, and other necessary parts.
  * Model architecture: layer number/size/type, activation function, etc
  * Training objectives: loss function, optimizer, weight of each loss term, etc
  * Others: whether the model is pretrained, Monte Carlo simulation for uncertainty analysis, etc
  * The code of model should have classes of the model, functions of model training, model validation, etc.
  * If your model training is done outside of this notebook, please upload the trained model here and develop a function to load and test it.

### Model Architecture

An autoencoder represents a type of neural network designed for the purpose of reconstructing its input data, denoted as $x$. In its fundamental structure, it comprises an encoder function, denoted as $f_\phi(⋅)$, and a decoder function, denoted as $f_\theta′(⋅)$, with $\phi$ and $\theta$ serving as the parameters associated with the encoder and decoder functions, respectively.
The training objective of an autoencoder is to minimize the disparity between the original input $x$ and its reconstructed counterpart $x′$. This discrepancy, typically quantified using a reconstruction loss metric such as squared error, can be mathematically expressed as $L(x, x')=||x-x'||^2=||x-f_\theta'(f_\phi(x))||^2$.

In this project, we focus on the utilization of a trained autoencoder to obtain a lower-dimensional latent representation $z = f_\phi(x)$ of the input. There are four autoencoders that we incorporate as below.

1. Shallow Autoencoder (SAE): a fully connected encoder connecting the input layer to the latent layer, and a decoder producing the reconstructed input $x′$ by combining the outputs of the latent layer using weighted sums, with both the latent and output layers utilizing a linear activation function.

2. Deep Autoencoder (DAE): enhanced SAE model by inserting hidden layers with Rectified Linear Unit (ReLu) activation function and Glorot uniform initializer between the input and latent layers, maintaining an equal number of hidden layers (either one or two layers) in both the encoder and decoder sections.

3. Variational autoencoder (VAE): it learns probabilistic representations $z$ by approximating the true posterior distribution with $q_\phi(z|x)$ assuming a Gaussian distribution. The encoder encodes the means and variances of the Gaussian distribution, allowing sampling of latent representation $z$. This sampled representation is then fed into the decoder network to generate the reconstructed input $x′ \sim g_\theta(x|z)$.

4. convolutional autoencoder (CAE): equipped with convolutional layers where each unit is connected locally to the previous layer. The layers consist of multiple filters with weights for convolution operations. We employed ReLu activation, Glorot uniform initializer, and avoided pooling layers to prevent excessive generalization. The $n$-dimensional input vector was reshaped into a squared image of size $d \times d \times 1$, where $d = ⌊ \sqrt{n} ⌋ + 1$.

After getting the low-dimentional representations of profiles, we build classification models.

1. Support Vector Machine (SVM): a grid search using SVM is conducted to explore the hyper-parameter space. We considered both radial basis function (RBF) and linear kernels, adjusting penalty parameter C and kernel coefficient gamma for RBF.

2. Random Forest (RF): We examine two criteria, Gini impurity and information gain, for selecting features to split a node in a decision tree. The maximum number of features considered for the best split at each node was determined using the square root of the sample size and the logarithm to base 2 of the sample size.

3. Multi-Layer Perceptron (MLP): ReLu activations were used for the hidden layers, while the output layer employed sigmoid activation with a single unit. The number of units in the hidden layers was set to half of the preceding layer, excluding the first hidden layer.

The overall workflow is demonstrated in the Figure 1 below.

![]("assets/Model_Figure1.png")

### Training Objectives
- Split each dataset into a training set, validation set, and test set (64% training, 16% validation, and 20% test).
- Exclude the test set from model training.
- Implement early-stopping strategy: train models on the training set, compute reconstruction loss for the validation set after each epoch, stop training if no improvement in validation loss is observed for 20 epochs.
- Select the model with the lowest validation loss as the best model.
Utilize mean squared error as the reconstruction loss metric.
- Apply adaptive moment estimation (Adam) optimizer with default parameters (learning rate: 0.001, epsilon: 1e-07) as specified in the original paper.
- Utilize the encoder part of the best model to generate low-dimensional representations of microbiome data for subsequent disease prediction tasks.


### Evaluations

- Conduct 5-fold cross-validation on the reduced training set. This involves dividing the training set into five subsets, using four subsets for training and one for validation in each fold. This is to vary hyper-parameters and explore different combinations to find the best performing configuration.
- Evaluate the performance of the models using the area under the receiver operating characteristics curve (AUC). This metric assesses the model's ability to distinguish between different classes and is commonly used for classification tasks.
- Train a final classification model using the entire training set and the best hyper-parameter combination identified during cross-validation. This model aims to achieve optimal performance based on the selected configuration. Then test the final classification model on the separate test set, which was not used during training. This evaluation provides an unbiased assessment of the model's performance on unseen data.
- Repeat the entire procedure five times, each time using a different random partition seed to create new training, validation, and test sets. This helps account for potential variations in performance due to the specific data splits. Average the resulting AUC scores obtained from the five repetitions. This average serves as a summary metric to compare the performance of different models or approaches. It provides a more robust assessment by considering multiple iterations of the evaluation process.

### Autoencoders Definitions



### Train Models

### Run Experiments

# Results
In this section, you should finish training your model training or loading your trained model. That is a great experiment! You should share the results with others with necessary metrics and figures.

Please test and report results for all experiments that you run with:

*   specific numbers (accuracy, AUC, RMSE, etc)
*   figures (loss shrinkage, outputs from GAN, annotation or label of sample pictures, etc)



---


**Current Progress:**

Successfully run model with ae, vae, pca, cae with auc results.

**To-Do:**

Show figures for each (By Apr 21)

config ae test
loaded data from data/abundance/abundance_Cirrhosis.txt
X_train.shape:  (185, 10)
y_train.shape:  (185,)
X_test.shape:  (47, 10)
y_test.shape:  (47,)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 10)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 50)                  │             550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             510 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,060 (4.14 KB)

 Trainable params: 1,060 (4.14 KB)

...

[CV 4/5; 2/2] END dropout_rate=0.3, epochs=30, numHiddenLayers=2, numUnits=10;, score=0.734 total time=   8.1s
[CV 5/5; 2/2] START dropout_rate=0.3, epochs=30, numHiddenLayers=2, numUnits=10.


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[CV 5/5; 2/2] END dropout_rate=0.3, epochs=30, numHiddenLayers=2, numUnits=10;, score=0.711 total time=   8.0s


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best parameters set found on development set:

{'dropout_rate': 0.1, 'epochs': 30, 'numHiddenLayers': 2, 'numUnits': 10}
Accuracy metrics
AUC, ACC, Recall, Precision, F1_score, time-end, runtime(sec), classfication time(sec), best hyper-parameter
[0.5779, 0.5532, 0.9167, 0.5366, 0.6769, '2024-04-16 09:30:42.324221', 138.25, 97.49, "{'dropout_rate': 0.1, 'epochs': 30, 'numHiddenLayers': 2, 'numUnits': 10}"]


## Model comparison

**To-Do:**

Train model in full epoches and compare results. (By Apr 28)
- we can compare the results of the different models in terms of accuracy, AUC, etc. And,
- the time taken to train each model
- the best hyperparameters found for each model

# Discussion

In this section,you should discuss your work and make future plan. The discussion should address the following questions:
  * Make assessment that the paper is reproducible or not.
  * Explain why it is not reproducible if your results are kind negative.
  * Describe “What was easy” and “What was difficult” during the reproduction.
  * Make suggestions to the author or other reproducers on how to improve the reproducibility.
  * What will you do in next phase.

---

**To-Do:**

Add discussion. (By Apr 28)

- discuss the reproducibility of the paper and the ease of reproducing the results.
- discuss what was easy and what was difficult during the reproduction.
- make suggestions to the author or other reproducers on how to improve the reproducibility.
- discuss the results of the model comparison and the implications of the results.
- discuss the limitations of the study and the potential future work.



# References

1. Cho, I., & Blaser, M. J. (2012). The human microbiome: at the interface of health and disease. Nature Reviews Genetics, 13(4), 260-270.
2. Huttenhower, C. et al. Structure, function and diversity of the healthy human microbiome. nature 486, 207 (2012).
3. McQuade, J. L., Daniel, C. R., Helmink, B. A. & Wargo, J. A. Modulating the microbiome to improve therapeutic response in cancer. The Lancet Oncology 20, e77–e91 (2019).
4. Eloe-Fadrosh, E. A. & Rasko, D. A. The human microbiome: from symbiosis to pathogenesis. Annual review of medicine 64, 145–163 (2013).
5. Scholz, M. et al. Strain-level microbial epidemiology and population genomics from shotgun metagenomics. Nature methods 13, 435 (2016).
6. Truong, D. T. et al. MetaPhlAn2 for enhanced metagenomic taxonomic profiling. Nature methods 12, 902 (2015).
7. Oh, M., & Zhang, L. (2020). DeepMicro: deep representation learning for disease prediction based on microbiome data. Scientific reports, 10(1), 6026.
8. Pasolli, E., Truong, D. T., Malik, F., Waldron, L. & Segata, N. Machine learning meta-analysis of large metagenomic datasets: tools and biological insights. PLoS computational biology 12, e1004977 (2016).
9. Cawley, G. C. & Talbot, N. L. On over-fitting in model selection and subsequent selection bias in performance evaluation. Journal of Machine Learning Research 11, 2079–2107 (2010).
10. Varma, S. & Simon, R. Bias in error estimation when using cross-validation for model selection. BMC bioinformatics 7, 91 (2006).
